In [ ]:
%load_ext autoreload
%autoreload 2
import torch
import meshplot as mp
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import altair as alt


from brepmatching.data import convert_cache

In [ ]:
geo = torch.load('../data/GeoFinal.pt')
both = torch.load('../data/BothFinal.pt')
topo = torch.load('../data/TopoFinal.pt')

In [ ]:
geo[0]['data'].left_faces.device

In [ ]:
geo_converted = convert_cache(geo)
topo_converted = convert_cache(topo)
both_converted = convert_cache(both)

In [ ]:
def get_stats_converted(cache):
    lost_count = []
    lost_pct = []
    face_max = []
    edge_max = []
    vert_max = []

    for data in cache['preprocessed_data']:
        has_baseline = data.has_onshape_baseline[0].item()
        if has_baseline:
            baseline_lost = data.n_onshape_baseline_unmatched[0].item()
            baseline_face = data.os_bl_faces_matches.shape[1]
            baseline_edge = data.os_bl_edges_matches.shape[1]
            baseline_vert = data.os_bl_vertices_matches.shape[1]
            baseline_total = baseline_face + baseline_edge + baseline_vert
            pct_lost = 0. if baseline_lost == 0 else baseline_lost / (baseline_total + baseline_lost)

            face_max.append(
                max(
                    data.left_faces.abs().max().item() if len(data.left_faces) > 0 else 0., 
                    data.right_faces.abs().max().item()if len(data.right_faces) > 0 else 0.
                )
            )

            edge_max.append(
                max(
                    data.left_edges.abs().max().item() if len(data.left_edges) > 0 else 0.,
                    data.right_edges.abs().max().item()if len(data.right_edges) > 0 else 0.
                )
            )

            vert_max.append(
                max(
                    data.left_vertices.abs().max().item() if len(data.left_vertices) > 0 else 0., 
                    data.right_vertices.abs().max().item() if len(data.right_vertices) > 0 else 0.
                )
            )

            lost_count.append(baseline_lost)
            lost_pct.append(pct_lost)
            
    lost_count = np.array(lost_count)
    lost_pct = np.array(lost_pct)
    face_max = np.array(face_max)
    edge_max = np.array(edge_max)
    vert_max = np.array(vert_max)

    return {
        'lost_count': lost_count,
        'lost_pct': lost_pct,
        'face_max': face_max,
        'edge_max': edge_max,
        'vert_max': vert_max
    }

In [ ]:
geo_stats = get_stats_converted(geo_converted)
topo_stats = get_stats_converted(topo_converted)
both_stats = get_stats_converted(both_converted)

def filter_count(stats, face_thresh = 200, edge_thresh = 200, vert_thresh = 5):
    filt_lost = stats['lost_count'] == 0
    filt_face = stats['face_max'] < face_thresh
    filt_edge = stats['edge_max'] < edge_thresh
    filt_vert = stats['vert_max'] < vert_thresh

    filt = filt_lost & filt_face & filt_edge & filt_vert

    return filt.sum()

def filter_records(gt, vt):
    return [{
        'geo_thresh': gt,
        'vert_thresh': vt,
        'ds':'geo',
        'count': filter_count(geo_stats, gt, gt, vt),
    },
    {
        'geo_thresh': gt,
        'vert_thresh': vt,
        'ds':'topo',
        'count': filter_count(topo_stats, gt, gt, vt)
    },
    {
        'geo_thresh': gt,
        'vert_thresh': vt,
        'ds':'both',
        'count': filter_count(both_stats, gt, gt, vt)
    }
    ]

f_records = []
for gt in np.linspace(0,1000,1001):
    for vt in [4]:#np.linspace(0,20,21):
        f_records += filter_records(gt,vt)
df = pd.DataFrame.from_records(f_records)


In [ ]:
alt.data_transformers.disable_max_rows()

alt.Chart(df).mark_line().encode(
    x='geo_thresh',
    y='count',
    color='vert_thresh:O',
    column='ds'
)

In [ ]:
np.abs(np.abs(geo_stats['face_max'][geo_stats['face_max'] > 400]) - 500) < .1

In [ ]:
len([i for i,d in enumerate(geo) if d['reason'] == 'ok'])

In [ ]:
len([i for i,d in enumerate(both) if d['reason'] == 'ok'])

In [ ]:
geo[0]['data'].has_onshape_baseline

In [ ]:
def get_stats(ds):
    lost_count = []
    lost_pct = []
    face_max = []
    edge_max = []
    vert_max = []

    for d in ds:
        if d['reason'] == 'ok':
            data = d['data']
            has_baseline = data.has_onshape_baseline[0].item()
            if has_baseline:
                baseline_lost = data.n_onshape_baseline_unmatched[0].item()
                baseline_face = data.os_bl_faces_matches.shape[1]
                baseline_edge = data.os_bl_edges_matches.shape[1]
                baseline_vert = data.os_bl_vertices_matches.shape[1]
                baseline_total = baseline_face + baseline_edge + baseline_vert
                pct_lost = 0. if baseline_lost == 0 else baseline_lost / (baseline_total + baseline_lost)

                face_max.append(
                    max(
                        data.left_faces.abs().max().item() if len(data.left_faces) > 0 else 0., 
                        data.right_faces.abs().max().item()if len(data.right_faces) > 0 else 0.
                    )
                )

                edge_max.append(
                    max(
                        data.left_edges.abs().max().item() if len(data.left_edges) > 0 else 0.,
                        data.right_edges.abs().max().item()if len(data.right_edges) > 0 else 0.
                    )
                )

                vert_max.append(
                    max(
                        data.left_vertices.abs().max().item() if len(data.left_vertices) > 0 else 0., 
                        data.right_vertices.abs().max().item() if len(data.right_vertices) > 0 else 0.
                    )
                )

                lost_count.append(baseline_lost)
                lost_pct.append(pct_lost)
    lost_count = np.array(lost_count)
    lost_pct = np.array(lost_pct)
    face_max = np.array(face_max)
    edge_max = np.array(edge_max)
    vert_max = np.array(vert_max)

    return {
        'lost_count': lost_count,
        'lost_pct': lost_pct,
        'face_max': face_max,
        'edge_max': edge_max,
        'vert_max': vert_max
    }


In [ ]:
for d in geo:
    if d['reason'] == 'ok':
        d['geo']

In [ ]:
foobar = [
    {'a':{'b':np.array([1,0,0])}}
]

In [ ]:
geo[0]

In [ ]:
geo_stats = get_stats(geo)
topo_stats = get_stats(topo)
both_stats = get_stats(both)

In [ ]:
def filter_count(stats, face_thresh = 200, edge_thresh = 200, vert_thresh = 5):
    filt_lost = stats['lost_count'] == 0
    filt_face = stats['face_max'] < face_thresh
    filt_edge = stats['edge_max'] < edge_thresh
    filt_vert = stats['vert_max'] < vert_thresh

    filt = filt_lost & filt_face & filt_edge & filt_vert

    return filt.sum()

In [ ]:
def filter_count(stats, face_thresh = 200, edge_thresh = 200, vert_thresh = 5):
    filt_lost = stats['lost_count'] == 0
    filt_face = stats['face_max'] < face_thresh
    filt_edge = stats['edge_max'] < edge_thresh
    filt_vert = stats['vert_max'] < vert_thresh

    filt = filt_lost & filt_face & filt_edge & filt_vert

    return filt.sum()

def filter_records(gt, vt):
    return [{
        'geo_thresh': gt,
        'vert_thresh': vt,
        'ds':'geo',
        'count': filter_count(geo_stats, gt, gt, vt),
    },
    {
        'geo_thresh': gt,
        'vert_thresh': vt,
        'ds':'topo',
        'count': filter_count(topo_stats, gt, gt, vt)
    },
    {
        'geo_thresh': gt,
        'vert_thresh': vt,
        'ds':'both',
        'count': filter_count(both_stats, gt, gt, vt)
    }
    ]

f_records = []
for gt in np.linspace(0,1000,1001):
    for vt in np.linspace(0,20,21):
        f_records += filter_records(gt,vt)

In [ ]:
df.columns

In [ ]:
expert['preprocessed_data'][0].left_vertices

In [ ]:
from brepmatching.data import ParallelPreprocessor, load_data

In [ ]:
expert = load_data('../data/ExpertDataWithBaseline.zip', None)

In [ ]:
expert_face_max = []
expert_edge_max = []
expert_vert_max = []
for data in expert['preprocessed_data']:
    expert_face_max.append(
        max(
            data.left_faces[:,:15].abs().max().item() if len(data.left_faces) > 0 else 0., 
            data.right_faces[:,:15].abs().max().item()if len(data.right_faces) > 0 else 0.,
            data.left_faces[:,18:].abs().max().item() if len(data.left_faces) > 0 else 0., 
            data.right_faces[:,18:].abs().max().item()if len(data.right_faces) > 0 else 0.
        )
    )

    expert_edge_max.append(
        max(
            data.left_edges.abs().max().item() if len(data.left_edges) > 0 else 0.,
            data.right_edges.abs().max().item()if len(data.right_edges) > 0 else 0.
        )
    )

    expert_vert_max.append(
        max(
            data.left_vertices.abs().max().item() if len(data.left_vertices) > 0 else 0., 
            data.right_vertices.abs().max().item() if len(data.right_vertices) > 0 else 0.
        )
    )

expert_face_max = np.array(expert_face_max)
expert_edge_max = np.array(expert_edge_max)
expert_vert_max = np.array(expert_vert_max)

In [ ]:
f = expert['preprocessed_data'][15].left_edges

for i in range(f.shape[0]):
    for j in range(f.shape[1]):
        if f[i,j] == 500:
            print(f'{i},{j}')

In [ ]:
(f == 500).sum()

In [ ]:
expert['original_index'][15]

In [ ]:
plt.hist(expert_edge_max)

In [ ]:
(geo_stats['lost_count'] > 0).sum()

In [ ]:
(topo_stats['lost_count'] > 0).sum()

In [ ]:
(both_stats['lost_count'] > 0).sum()

In [ ]:
plt.hist(geo_stats['vert_max'], range(0,10))

In [ ]:
plt.hist(topo_stats['face_max'], range=(0,1000))

In [ ]:
plt.hist(both_stats['face_max'], range=(0,300))

In [ ]:
face_max = geo_stats['face_max']

In [ ]:
large_parts = np.arange(len(face_max))[((face_max > 480) & (face_max < 510))]

In [ ]:
data.left_V.max(dim=0)

In [ ]:
data.left_V.min(dim=0)

In [ ]:
f = (geo[large_parts[0]]['data'].left_faces)

for i in range(f.shape[0]):
    for j in range(f.shape[1]):
        if f[i,j] == -500:
            print(f'{i},{j}')

In [ ]:
(f == -500)

In [ ]:
big_stats = []
for i in large_parts:
    data = geo[i]['data']
    if data is None:
        continue
    V = data.left_V.numpy()
    center = (V.min(axis=0) + V.max(axis=0)) / 2
    size = V.max(axis=0) - V.min(axis=0)
    face_max = data.left_faces.abs().max()
    edge_max = data.left_edges.abs().max()
    vert_max = data.left_vertices.abs().max() if len(data.left_vertices) > 0 else 0
    big_stats.append({
        'i':i,
        'center':center,
        'size':size,
        'face_max':face_max,
        'edge_max':edge_max,
        'vert_max':vert_max
    })

In [ ]:
big_stats

In [ ]:
i = large_parts[25]

data = geo[i]['data']

mp.plot(data.left_V.numpy(), data.left_F.T.numpy())